In [1]:
import tensorflow as tf

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

model = Sequential([
    Dense(32, input_shape=(784,)),
    Activation('relu'),
    Dense(10),
    Activation('softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                25120     
_________________________________________________________________
activation (Activation)      (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                330       
_________________________________________________________________
activation_1 (Activation)    (None, 10)                0         
Total params: 25,450
Trainable params: 25,450
Non-trainable params: 0
_________________________________________________________________


In [3]:
model = Sequential()
model.add(Dense(32, input_dim=784))
model.add(Activation('relu'))
model.add(Dense(10))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 32)                25120     
_________________________________________________________________
activation_2 (Activation)    (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                330       
_________________________________________________________________
activation_3 (Activation)    (None, 10)                0         
Total params: 25,450
Trainable params: 25,450
Non-trainable params: 0
_________________________________________________________________


### specifying input shape

* input_shape=tuple (batch not included)
* input_dim=int (2D) input_length=int (3D)
* (optional) batch_size=32


### compilation

3 arguments

* optimizer: 'rmsprop' | 'adagrad' | ...
* loss: 'categorical_crossentropy' | 'mse' | ...
* metrics: 웬만해선 ['accuracy']일 것

In [6]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.compile(optimizer='rmsprop',
              loss='mse')

# Custom Metric

import tensorflow.keras.backend as K

def mean_pred(y_true, y_pred):
    return K.mean(y_pred)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', mean_pred])

### Training

In [17]:
import numpy as np

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(1, activation='sigmoid'))

model.compile(
    optimizer='rmsprop',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

train_X = np.random.random((1000, 100))
train_y = np.random.randint(2, size=(1000, 1))

model.fit(train_X, train_y, epochs=5, batch_size=32)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 487us/sample - loss: 0.6991 - accuracy: 0.5220
Epoch 2/5
1000/1000 [==============================] - 0s 121us/sample - loss: 0.6961 - accuracy: 0.5280
Epoch 3/5
1000/1000 [==============================] - 0s 129us/sample - loss: 0.6884 - accuracy: 0.5430
Epoch 4/5
1000/1000 [==============================] - 0s 135us/sample - loss: 0.6847 - accuracy: 0.5560
Epoch 5/5
1000/1000 [==============================] - 0s 134us/sample - loss: 0.6836 - accuracy: 0.5660


In [18]:
import tensorflow.keras as keras

model = Sequential()
model.add(Dense(32, activation='relu', input_dim=100))
model.add(Dense(10, activation='softmax'))

model.compile(
    optimizer='rmsprop',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

train_X = np.random.random((1000, 100))
train_y = np.random.randint(10, size=(1000, 1))

one_hot_train_y = keras.utils.to_categorical(train_y, num_classes=10)

model.fit(train_X, one_hot_train_y, epochs=5, batch_size=32)

Train on 1000 samples
Epoch 1/5
1000/1000 [==============================] - 0s 419us/sample - loss: 2.3685 - accuracy: 0.0960
Epoch 2/5
1000/1000 [==============================] - 0s 112us/sample - loss: 2.3172 - accuracy: 0.1110
Epoch 3/5
1000/1000 [==============================] - 0s 113us/sample - loss: 2.3030 - accuracy: 0.1110
Epoch 4/5
1000/1000 [==============================] - 0s 120us/sample - loss: 2.2953 - accuracy: 0.1200
Epoch 5/5
1000/1000 [==============================] - 0s 116us/sample - loss: 2.2856 - accuracy: 0.1320


### Functional API

* layer: tensor -> tensor
* Model(input tensor, output tensor)


In [31]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

inputs = Input(shape=(784, ))

output_1 = Dense(64, activation='relu')(inputs)
output_2 = Dense(64, activation='relu')(output_1)
predictions = Dense(10, activation='softmax')(output_2)

model = Model(inputs=inputs, outputs=predictions)

model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         [(None, 784)]             0         
_________________________________________________________________
dense_31 (Dense)             (None, 64)                50240     
_________________________________________________________________
dense_32 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_33 (Dense)             (None, 10)                650       
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


model도 layer처럼 tensor -> tensor 타입이고, trained weight까지 사용해서 변환이 가능하다.

In [34]:
x = Input(shape=(784,))
y = model(x)

In [35]:
from tensorflow.keras.layers import TimeDistributed
# TimeDistributed: 첫번째 dimension에 대해서 model을 map 해주는 functional

input_sequences = Input(shape=(20, 784)) # 여기서 20은 time dimension (iteration)
processed_sequences = TimeDistributed(model)(input_sequences)

Model(input_sequences, processed_sequences).summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 20, 784)]         0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 20, 10)            55050     
Total params: 55,050
Trainable params: 55,050
Non-trainable params: 0
_________________________________________________________________


In [36]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
np.random.seed(0)

main_input = Input(shape=(100,), dtype='int32', name='main_input')
x = Embedding(output_dim=512, input_dim=10000, input_length=100)(main_input)
lstm_out = LSTM(32)(x)
Model(main_input, lstm_out).summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      [(None, 100)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 512)          5120000   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                69760     
Total params: 5,189,760
Trainable params: 5,189,760
Non-trainable params: 0
_________________________________________________________________


In [37]:
aux_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)

In [39]:
aux_input = Input(shape=(5,), name='aux_input')
x = keras.layers.concatenate([lstm_out, aux_input])
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)
main_output = Dense(1, activation='sigmoid', name='main_output')(x)

In [40]:
model = Model(inputs=[main_input, aux_input], outputs=[main_output, aux_output])
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 512)     5120000     main_input[0][0]                 
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 32)           69760       embedding[0][0]                  
__________________________________________________________________________________________________
aux_input (InputLayer)          [(None, 5)]          0                                            
____________________________________________________________________________________________

In [42]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              loss_weights=[1., 0.2])

headline_data = np.round(np.abs(np.random.rand(12, 100) * 100))
additional_data = np.random.randn(12, 5)
headline_labels = np.random.randn(12, 1)
additional_labels = np.random.randn(12, 1)
model.fit([headline_data, additional_data], [headline_labels, additional_labels],
          epochs=50, batch_size=32)

Train on 12 samples
Epoch 1/50
12/12 [==============================] - 2s 161ms/sample


UnknownError:  [_Derived_]  Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[model_5/lstm/StatefulPartitionedCall]]
	 [[Reshape_22/_46]] [Op:__inference_distributed_function_11145]

Function call stack:
distributed_function -> distributed_function -> distributed_function


In [43]:
model.compile(optimizer='rmsprop',
              loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
              loss_weights={'main_output': 1., 'aux_output': 0.2})

# And trained it via:
model.fit({'main_input': headline_data, 'aux_input': additional_data},
          {'main_output': headline_labels, 'aux_output': additional_labels},
          epochs=50, batch_size=32)

Train on 12 samples
Epoch 1/50
12/12 [==============================] - 2s 154ms/sample


UnknownError:  [_Derived_]  Fail to find the dnn implementation.
	 [[{{node CudnnRNN}}]]
	 [[model_5/lstm/StatefulPartitionedCall]]
	 [[Reshape_22/_46]] [Op:__inference_distributed_function_14475]

Function call stack:
distributed_function -> distributed_function -> distributed_function
